In [93]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/xiaoqin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [110]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiaoqin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [111]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/xiaoqin/nltk_data...


True

In [112]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/xiaoqin/nltk_data...


True

In [94]:
from sklearn.feature_extraction.text import CountVectorizer

In [108]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string
import re

In [128]:
train=pd.read_csv('twittertrain.csv')
test=pd.read_csv('twittertest.csv')

In [129]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [130]:
print(train.shape)
test.shape

(7613, 5)


(3263, 4)

In [131]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [132]:
train=train.drop(columns=['id','keyword','location'])
train
            

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [133]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [134]:
feat=train[['text']]
label=train[['target']]

In [135]:
ps=PorterStemmer()
wordnet=WordNetLemmatizer()

In [136]:
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [137]:
def preprocessing(data):
    corpus=[]
    for i in range(len(data)):
        #remove urls
        tweet=re.sub(r'http\S+',' ',data[i])
        
        #remove html tags
        tweet=re.sub(r'<.*?>',' ',tweet)
        tweet=re.sub('@\w+([-.]\w+)*',' ',tweet)
        tweet=re.sub('&\w+([-.]\w+)*',' ',tweet)
        
        #remove digits
        tweet=re.sub(r'\d+',' ',tweet)
        
        #remove hashtags
        tweet=re.sub(r'#\w+',' ',tweet)
        #remove punctuations and numbers
        review=re.sub('[^a-zA-Z]',' ',tweet)
        review=review.lower()
        review=review.split()
        
        #review=expand_contractions(review)
        lemma=WordNetLemmatizer()
        review=[lemma.lemmatize(word) for word in review if word not in all_stopwords]
        review=' '.join(review)
        corpus.append(review)
    return corpus

In [138]:
feat.head()

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [139]:
feat['text']=preprocessing(feat['text'])

/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_65169/386183970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat['text']=preprocessing(feat['text'])


In [140]:
feat.head()

,text
0,deed reason may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked shelter place notified officer ...
3,people receive evacuation order california
4,got sent photo ruby smoke pours school


In [142]:
#Model Building
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [145]:
#preprocessing the text data
test=test.drop(columns=['id','keyword','location'])
test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [146]:
test['text']=preprocessing(test['text'])
test.head()

,text
0,happened terrible car crash
1,heard different city stay safe everyone
2,forest fire spot pond goose fleeing across str...
3,apocalypse lighting
4,typhoon soudelor kill china taiwan


In [147]:
test.shape

(3263, 1)

In [148]:
tfidf=TfidfVectorizer(min_df=1,stop_words='english')

xtrain_feat=tfidf.fit_transform(feat.text)
xtest_feat=tfidf.transform(test.text)

ytrain=label.astype("int")

In [ ]:
sklearn.model_selection.Train_test_split

In [197]:
tfidf.transform(['this is a tweet', 'is this a tweet']).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
word embedding

In [ ]:
king [1, 500, 4, 100, 10]
queen[1, 500, 4, 100, 20]
man  [25, 400, 4, 67, 10]
woman[25, 400, 4, 67, 20]

king-man+woman [1, 500, 4, 100, 20]
word embedding
word2vec: CBOW skip-gram

In [149]:
print(xtrain_feat)

  (0, 3826)	0.5404321307842656
  (0, 260)	0.5091998116723592
  (0, 8126)	0.3957041518922832
  (0, 2463)	0.5404321307842656
  (1, 1428)	0.4041746272133468
  (1, 8722)	0.5013434889003477
  (1, 8540)	0.5013434889003477
  (1, 5576)	0.3622437862730083
  (1, 6734)	0.3216119529304211
  (1, 3819)	0.31511501714498075
  (2, 3418)	0.2516652226079931
  (2, 7111)	0.22037114836821514
  (2, 3343)	0.2102667513754822
  (2, 6993)	0.22843460285459039
  (2, 6883)	0.324702456667253
  (2, 7536)	0.45686920570918077
  (2, 8995)	0.5722307285742921
  (2, 537)	0.271874016593459
  (2, 8344)	0.2760109672944131
  (3, 1401)	0.36584391607961997
  (3, 8143)	0.6216887930842094
  (3, 7395)	0.32808871029845726
  (3, 7111)	0.44128642622120473
  (3, 3343)	0.4210526829609798
  (4, 8776)	0.30373294586187666
  :	:
  (7610, 4482)	0.551138418251355
  (7610, 10676)	0.49411136471938033
  (7610, 5516)	0.47780029676663055
  (7610, 10838)	0.4730827407465482
  (7611, 9730)	0.2944433018935361
  (7611, 7659)	0.30434876395084237
  (7611

In [169]:
test_copy=pd.read_csv("twittertest.csv")

In [170]:
#training the logistic regression model
log=LogisticRegression()
log.fit(xtrain_feat,ytrain)

/Users/xiaoqin/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [171]:
test_copy['target']=log.predict(xtest_feat)

In [172]:
test_copy=test_copy.drop(columns=['keyword','location'])

In [173]:
test_copy.set_index("id",inplace=True)
test_copy

,text,target
id,,
0,Just happened a terrible car crash,1
2,"Heard about #earthquake is different cities, s...",0
3,"there is a forest fire at spot pond, geese are...",1
9,Apocalypse lighting. #Spokane #wildfires,0
11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...
10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
10865,Storm in RI worse than last hurricane. My city...,1
10868,Green Line derailment in Chicago http://t.co/U...,1


In [174]:
#save to csv
test_copy.to_csv("spamclassifier.csv")

In [183]:
#using naive bayes
test_copy1=pd.read_csv("twittertest.csv")

In [184]:
from sklearn.naive_bayes import MultinomialNB

In [185]:
nb=MultinomialNB()
nb.fit(xtrain_feat,ytrain)

/Users/xiaoqin/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [186]:
test_copy1['target']=nb.predict(xtest_feat)

In [187]:
test_copy1=test_copy1.drop(columns=['keyword','location'])

In [188]:
test_copy1.set_index("id",inplace=True)
test_copy1

,text,target
id,,
0,Just happened a terrible car crash,1
2,"Heard about #earthquake is different cities, s...",0
3,"there is a forest fire at spot pond, geese are...",1
9,Apocalypse lighting. #Spokane #wildfires,0
11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...
10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
10865,Storm in RI worse than last hurricane. My city...,1
10868,Green Line derailment in Chicago http://t.co/U...,1


In [189]:
test_copy1.to_csv("spamclassifier_naivebayes.csv")

In [190]:
test_copy.target.value_counts()

0    2167
1    1096
Name: target, dtype: int64

In [191]:
test_copy1.target.value_counts()

0    2167
1    1096
Name: target, dtype: int64

In [98]:
#vectorizer=CountVectorizer()

In [99]:
#for row in text['text']:
#    tokens = word_tokenize(row.lower())
#    print(tokens)
#print(tokens)

['just', 'happened', 'a', 'terrible', 'car', 'crash']
['heard', 'about', '#', 'earthquake', 'is', 'different', 'cities', ',', 'stay', 'safe', 'everyone', '.']
['there', 'is', 'a', 'forest', 'fire', 'at', 'spot', 'pond', ',', 'geese', 'are', 'fleeing', 'across', 'the', 'street', ',', 'i', 'can', 'not', 'save', 'them', 'all']
['apocalypse', 'lighting', '.', '#', 'spokane', '#', 'wildfires']
['typhoon', 'soudelor', 'kills', '28', 'in', 'china', 'and', 'taiwan']
['we', "'re", 'shaking', '...', 'it', "'s", 'an', 'earthquake']
['they', "'d", 'probably', 'still', 'show', 'more', 'life', 'than', 'arsenal', 'did', 'yesterday', ',', 'eh', '?', 'eh', '?']
['hey', '!', 'how', 'are', 'you', '?']
['what', 'a', 'nice', 'hat', '?']
['fuck', 'off', '!']
['no', 'i', 'do', "n't", 'like', 'cold', '!']
['nooooooooo', '!', 'do', "n't", 'do', 'that', '!']
['no', 'do', "n't", 'tell', 'me', 'that', '!']
['what', 'if', '?', '!']
['awesome', '!']
['birmingham', 'wholesale', 'market', 'is', 'ablaze', 'bbc', 'news